In [37]:
!pip install qiskit
!pip install qiskit_experiments
!pip install azure-quantum[qiskit]==0.23.201228b1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:

from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = "/subscriptions/33e6f75e-0499-425c-8b7d-9f593dde82b6/resourceGroups/AQET/providers/Microsoft.Quantum/Workspaces/CHEM560",
    location = "westus")


from IPython.display import Image
from IPython.core.display import HTML 

import qiskit

from qiskit.quantum_info import DensityMatrix
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.framework.experiment_data import ExperimentData
import qiskit.quantum_info as qi
from qiskit_experiments.library import StateTomography

from qiskit import QuantumCircuit, assemble, Aer
from math import pi, sqrt
from qiskit.visualization import plot_bloch_multivector, plot_histogram, plot_state_city

import numpy as np



In [2]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- ionq.qpu
- ionq.simulator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim


In [3]:
ionq_simulator_backend = provider.get_backend("ionq.simulator")
ionq_qpu_backend = provider.get_backend("ionq.qpu")
aer_simulator_backend = Aer.get_backend('aer_simulator')

# CHEM 560 Problem Set 5


###### Implementing the UCC ansatz on quantum computers & VQE to get the energy for $H_2$

**1. First, define helper functions**

In [4]:
def get_gi_energy(result,g):
    count1 = 0.0
    count0 = 0.0
    if ('0' in result.get_counts()):
        count0 = result.get_counts()['0']

    if ('1' in result.get_counts()):
        count1 = result.get_counts()['1']

    prob1 = np.sqrt(count1/(count1+count0))
    prob0 = np.sqrt(count0/(count1+count0))
    expectation_val = prob0**2 - prob1**2
    energy = g*(expectation_val)

    return energy

In [5]:
def get_total_energy(job_list):
    gi_energy = []
    g0 = 2.1868 
    cum_energy = 0.0
    g_para = [0.5449, -1.2870, +0.6719, +0.0798, +0.0798]
    print('g%d Energy: %15.7f Eh' % (0,g0))
    for i in range (5):
        Egi =  get_gi_energy(job_list[i].result(), g_para[i])
        gi_energy.append(Egi)
        cum_energy += Egi
        print('g%d Energy: %15.7f Eh' % (i+1,Egi))
    
    total_energy = cum_energy + g0

    print('Total Energy: %15.7f Eh' % total_energy)
    print('\n')
    return total_energy

In [6]:
def contruct_gi_circuit(theta, g_number):
    

    # Create initial state |01>
    circuit_gi = QuantumCircuit(2,1)
    circuit_gi.name = 'measurement for g' + str(g_number)
    circuit_gi.x(1)

    # Apply the Ansatz
    circuit_gi.ry(np.pi/2,0)
    circuit_gi.rx(3*np.pi/2,1)
    circuit_gi.cx(0,1)
    circuit_gi.rz(theta,1)
    circuit_gi.cx(0,1)
    circuit_gi.ry(3*np.pi/2,0)
    circuit_gi.rx(np.pi/2,1)

    # Obtain measurement
    if (g_number == 1 ):
        # g1 <I1 Sz0>
        circuit_gi.swap(0,1)   
    elif (g_number == 3):
        # g3 <Sz1 Sz0>
        circuit_gi.cx(0,1)
    elif (g_number == 4):
        # g4 <Sx1 Sx0>
        circuit_gi.cx(0,1)
        circuit_gi.h(0)
        circuit_gi.h(1)
    elif (g_number == 5):
        # g5 <Sy1 Sy0>
        circuit_gi.cx(0,1)
        circuit_gi.h(0)
        circuit_gi.sdg(0)
        circuit_gi.h(1)
        circuit_gi.sdg(1)

    # g2 <SZ1 I0>, so if g_number = 2, not need to do anything  
    
    circuit_gi.measure([1],[0])
    # circuit_gx.draw()
    return circuit_gi


In [7]:
def construct_circuit_list(theta):
    circuit_list = []
    for gi in range(1,6):
        circuit_list.append( contruct_gi_circuit(theta,gi) )
    return circuit_list

In [8]:
def procedural(theta,sim, nshots):
    global total_call
    total_call +=1

    t = theta[0]

    circuit_list = []

    circuit_list = construct_circuit_list(t)

    job_g1 = sim.run(circuit_list[0], shots = nshots)
    job_g2 = sim.run(circuit_list[1], shots = nshots)
    job_g3 = sim.run(circuit_list[2], shots = nshots)
    job_g4 = sim.run(circuit_list[3], shots = nshots)
    job_g5 = sim.run(circuit_list[4], shots = nshots)    
    

    job_list = [job_g1, job_g2, job_g3, job_g4, job_g5]

    return get_total_energy(job_list)

In [9]:
from scipy.linalg import expm
from scipy.optimize import minimize

In [10]:
#theta  = [+3.0546895]
theta  = [+3.0]
global total_call
total_call = 0

result = minimize(procedural,theta,args=(aer_simulator_backend, 20000),method='COBYLA',bounds=[(0,2*np.pi)],options={'disp':True,'maxiter':50})

theta  = result.x[0]
val    = result.fun
#it   = result.nit
print("Aer Simulator VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val))
#print("  [+] iterations: {:+2} ".format(it))

/usr/local/lib/python3.7/site-packages/scipy/optimize/_minimize.py:545: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


g0 Energy:       2.1868000 Eh
g1 Energy:      -0.5394510 Eh
g2 Energy:      -1.2749022 Eh
g3 Energy:      -0.6719000 Eh
g4 Energy:      -0.0005187 Eh
g5 Energy:       0.0003032 Eh
Total Energy:      -0.2996687 Eh


g0 Energy:       2.1868000 Eh
g1 Energy:      -0.3587077 Eh
g2 Energy:      -0.8350056 Eh
g3 Energy:      -0.6719000 Eh
g4 Energy:      -0.0001835 Eh
g5 Energy:       0.0005825 Eh
Total Energy:       0.3215857 Eh


g0 Energy:       2.1868000 Eh
g1 Energy:      -0.2295119 Eh
g2 Energy:      -0.5435001 Eh
g3 Energy:      -0.6719000 Eh
g4 Energy:      -0.0000399 Eh
g5 Energy:       0.0004150 Eh
Total Energy:       0.7422631 Eh


g0 Energy:       2.1868000 Eh
g1 Energy:      -0.4340129 Eh
g2 Energy:      -1.0302435 Eh
g3 Energy:      -0.6719000 Eh
g4 Energy:       0.0003032 Eh
g5 Energy:       0.0001516 Eh
Total Energy:       0.0510985 Eh


g0 Energy:       2.1868000 Eh
g1 Energy:      -0.5420665 Eh
g2 Energy:      -1.2803076 Eh
g3 Energy:      -0.6719000 Eh
g4 Energy:      -0.0

In [11]:
result.success

True

In [12]:
print(total_call)

18


In [13]:
#theta  = [+3.0546895]
theta  = [+3.0]
global total_call
total_call = 0

result = minimize(procedural,theta,args=(ionq_qpu_backend, 500),method='COBYLA',bounds=[(0,2*np.pi)],options={'disp':True,'maxiter':50})

theta  = result.x[0]
val    = result.fun
#it   = result.nit
print("Aer Simulator VQE: ")
print("  [+] theta:  {:+2.8} deg".format(theta))
print("  [+] energy: {:+2.8} Eh".format(val))
#print("  [+] iterations: {:+2} ".format(it))

g0 Energy:       2.1868000 Eh
...................g1 Energy:      -0.4707936 Eh
g2 Energy:      -1.2046320 Eh
g3 Energy:      -0.6235232 Eh
.....g4 Energy:      -0.0019152 Eh
.......g5 Energy:       0.0003192 Eh
Total Energy:      -0.1137448 Eh


g0 Energy:       2.1868000 Eh
...........................g1 Energy:      -0.4097648 Eh
g2 Energy:      -0.9678240 Eh
g3 Energy:      -0.6288984 Eh
g4 Energy:      -0.0041496 Eh
g5 Energy:      -0.0003192 Eh
Total Energy:       0.1758440 Eh


g0 Energy:       2.1868000 Eh
...........................................g1 Energy:      -0.1089800 Eh
g2 Energy:      -0.4427280 Eh
g3 Energy:      -0.5832092 Eh
g4 Energy:       0.0003192 Eh
g5 Energy:      -0.0057456 Eh
Total Energy:       1.0464564 Eh


g0 Energy:       2.1868000 Eh
.................g1 Energy:      -0.3574544 Eh
g2 Energy:      -0.9472320 Eh
g3 Energy:      -0.5832092 Eh
..g4 Energy:       0.0022344 Eh
.....g5 Energy:       0.0047880 Eh
Total Energy:       0.3059268 Eh


g0 Energy:     

I implemented the circuit and wrote a post-analysis function to calculate the energy. The optimizer will optimize the procedural fucntion, which builds the circuit, submits jobs, and does analysis to return a total energy.

Using the Aer simulator, I was able to get close to the optimal theta from any randomized radian. And the total energy is close to the true energy.

However, I wasn't able to make it work with QPU. The optimizer needs the previous jobs to finish so that it can do gradient calculations. This make the VQE process impossible to do in a a-sync manner. Due to the long waiting time on QPU, the kernal will eventually time out before any meaningful calculation is done. 